# 🇲🇬 Voambolana Malagasy — NLP Project

Ce notebook centralise la collecte de données et la préparation du modèle IA.

### 📑 Étapes d'utilisation :
1.  **Exécutez la cellule de configuration** ci-dessous pour connecter votre Drive et récupérer le code.
2.  **Lancez les scrapers** (Phases 1-4) pour collecter les paroles.
3.  **Nettoyez et Consolidez** (Phases 5-6) : Fusion Chansons + Bible + Wikipédia.
4.  **Fondations NLP** (Phase 7-10) : Tokenizer, Normalisation et Embeddings.
5.  **Applications** (Phase 12) : Testez le moteur de recherche et le correcteur.

## 🛠️ 1. Configuration (Drive + Git)

In [ ]:
from google.colab import drive
import os

# 1. Monter le Drive
print("🔗 Connexion à Google Drive...")
try:
    drive.mount('/content/drive', force_remount=True)
    print("✅ Drive monté avec succès !")
except Exception as e:
    print("❌ Erreur lors du montage du Drive.")

# 2. Se placer dans le dossier du projet
REPO_NAME = "nlp_malagasy"
BASE_PATH = "/content/drive/MyDrive/" + REPO_NAME

if not os.path.exists(BASE_PATH):
    %cd /content/drive/MyDrive
    !git clone https://github.com/franck504/nlp_malagasy.git

%cd $BASE_PATH
!git pull origin main

print(f"\n📍 Position actuelle : {os.getcwd()}")
!ls

# 🏗️ PARTIE 1 : COLLECTE DES DONNÉES (Scraping)

In [ ]:
!pip install requests beautifulsoup4
%cd $BASE_PATH/tononkira_rehetra

print("--- SCRAPING & MERGE ---")
!python 01_discover_artists.py --delay 1.0
!python 02_scrape_lyrics.py --delay 0.5 --artist-workers 4 --song-workers 10
!python 04_merge_corpus.py

# 🧹 PARTIE 2 : NETTOYAGE ET CONSOLIDATION

In [ ]:
%cd $BASE_PATH/tononkira_rehetra
print("--- CLEANING & WIKI ---")
!python 05_clean_corpus.py --input malagasy_lyrics_corpus.txt --output malagasy_lyrics_cleaned.txt
!python 10_download_wikipedia.py
!python 11_extract_wiki.py

print("\n--- CONSOLIDATION FINALE (Lyrics + Bible + Wiki) ---")
!python 06_consolidate_corpus.py --lyrics malagasy_lyrics_cleaned.txt --bible ../from_bible_json --wiki ../malagasy_wikipedia_raw.txt --output malagasy_corpus_v2_final.txt

# 🧠 PARTIE 3 : FONDATIONS NLP

In [ ]:
!pip install tokenizers gensim
%cd $BASE_PATH/tononkira_rehetra

print("--- TOKENIZATION & EMBEDDINGS ---")
!python 07_train_tokenizer.py --corpus malagasy_corpus_v2_final.txt --output tokenizer_mg
!python 09_normalize_corpus.py --input malagasy_corpus_v2_final.txt --output malagasy_corpus_normalized.txt
!python 08_train_embeddings.py --corpus malagasy_corpus_normalized.txt --output embeddings_mg

# 🚀 PARTIE 4 : APPLICATIONS (DÉMO)

### 🏗️ 1. Indexation Sémantique

> [!IMPORTANT]
> **BOUST TURBO (Recommandé sur Drive)** : Exécutez le script 13 d'abord pour regrouper les 14 000 fichiers en un seul. Cela accélère l'indexation de façon spectaculaire.

In [ ]:
%cd $BASE_PATH/tononkira_rehetra
print("🚀 Regroupement des chansons (Turbo Bundle)...")
!python 13_bundle_songs.py

In [ ]:
%cd $BASE_PATH/tononkira_rehetra
print("🏗️ Indexation sémantique (via le bundle)...")
!python 12_test_app.py --index

### 🔍 2. Tableau de Bord Interactif
Modifiez les paramètres ci-dessous pour tester l'intelligence du modèle.

In [ ]:
%cd $BASE_PATH/tononkira_rehetra

RECHERCHE_SENS = "Fitiavana sy fahatokiana"  # @param {type:"string"}
CORRECTION_MOT = "fityavana"               # @param {type:"string"}
EXPLORER_CONCEPT = "Antananarivo"          # @param {type:"string"}
STYLE_A_DETECTER = "Andriamanitra ô, tahio ny tany" # @param {type:"string"}

print(f"--- 🔍 RECHERCHE SÉMANTIQUE : {RECHERCHE_SENS} ---")
!python 12_test_app.py --query "{RECHERCHE_SENS}"

print(f"\n--- 📝 CORRECTION : {CORRECTION_MOT} ---")
!python 12_test_app.py --spell "{CORRECTION_MOT}"

print(f"\n--- 🧠 ANALYSE DE CONCEPT : {EXPLORER_CONCEPT} ---")
!python 12_test_app.py --explore "{EXPLORER_CONCEPT}"

print(f"\n--- 🎭 DÉTECTION DE STYLE ---")
!python 12_test_app.py --style "{STYLE_A_DETECTER}"

print(f"\n--- ⚖️ ANALOGIE ---")
!python 12_test_app.py --analogy lehilahy mpanjaka vehivavy